In [ ]:
!pip uninstall -y mediapipe
!pip install mediapipe==0.10.5

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.5/33.5 MB 19.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.1/162.1 kB 6.4 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 5.29.4
    Uninstalling protobuf-5.29.4:
      Successfully uninstalled protobuf-5.29.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-metadata 1.17.1 requires protobuf<6.0.0,>=4.25.2; python_version >= "3.11", but you have protobuf 3.20.3 which is incompatible.
grpcio-status 1.71.0 requires protobuf<6.0dev,>=5.26.1, but you have protobuf 3.20.3 which is incompatible.
ydf 0.11.0 requires protobuf<6.0.0,>=5.29.1, but you have protobuf 3.20.3 which is incompatible.


In [ ]:
import os
import cv2
import mediapipe as mp
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from google.colab import files
from tensorflow.keras import regularizers
from tensorflow.keras.losses import CategoricalCrossentropy


In [ ]:
IMG_SIZE = (224, 224)
BATCH_SIZE = 32
NUM_CLASSES = 36

In [ ]:
RAW_DIR  = "/content/drive/MyDrive/asl"
CROP_DIR = "/content/drive/MyDrive/asl_cropped"

In [ ]:
mp_hands = mp.solutions.hands.Hands(
    static_image_mode=True,
    max_num_hands=1,
    min_detection_confidence=0.5
)

In [ ]:
os.makedirs(CROP_DIR, exist_ok=True)
for label in os.listdir(RAW_DIR):
    os.makedirs(f"{CROP_DIR}/{label}", exist_ok=True)
    for fname in os.listdir(f"{RAW_DIR}/{label}"):
        img = cv2.imread(f"{RAW_DIR}/{label}/{fname}")
        if img is None:
            continue

        results = mp_hands.process(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
        h, w, _ = img.shape
        if results.multi_hand_landmarks:
            lm = results.multi_hand_landmarks[0]
            xs = [pt.x * w for pt in lm.landmark]
            ys = [pt.y * h for pt in lm.landmark]
            xmin = max(int(min(xs) - 20), 0)
            xmax = min(int(max(xs) + 20), w)
            ymin = max(int(min(ys) - 20), 0)
            ymax = min(int(max(ys) + 20), h)
            crop = img[ymin:ymax, xmin:xmax]
        else:
            crop = img

        crop = cv2.resize(crop, IMG_SIZE)
        cv2.imwrite(f"{CROP_DIR}/{label}/{fname}", crop)

mp_hands.close()

In [ ]:
train_datagen = ImageDataGenerator(
    rescale=1/255.,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    brightness_range=[0.8, 1.2],
    validation_split=0.2
)

In [ ]:
train_gen = train_datagen.flow_from_directory(
    CROP_DIR,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="categorical",
    subset="training"
)
val_gen = train_datagen.flow_from_directory(
    CROP_DIR,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="categorical",
    subset="validation"
)


Found 2012 images belonging to 36 classes.
Found 503 images belonging to 36 classes.


In [ ]:
base = tf.keras.applications.MobileNetV2(
    input_shape=IMG_SIZE + (3,),
    include_top=False, weights="imagenet"
)



base.trainable = True
fine_tune_at = 100
for layer in base.layers[:fine_tune_at]:
    layer.trainable = False

9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [ ]:
model = keras.Sequential([
    base,
    keras.layers.GlobalAveragePooling2D(),
    keras.layers.Dense(512,
        activation="relu",
        kernel_regularizer=regularizers.l2(0.01)
    ),
    keras.layers.BatchNormalization(),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(NUM_CLASSES, activation="softmax")
])

model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=1e-5),
    loss=CategoricalCrossentropy(label_smoothing=0.1),
    metrics=["accuracy"]
)

In [ ]:
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, min_lr=1e-7)

history = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=50,
    callbacks=[early_stopping, reduce_lr]
)


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/50
63/63 ━━━━━━━━━━━━━━━━━━━━ 752s 11s/step - accuracy: 0.0337 - loss: 12.1391 - val_accuracy: 0.0278 - val_loss: 11.2408 - learning_rate: 1.0000e-05
Epoch 2/50
63/63 ━━━━━━━━━━━━━━━━━━━━ 222s 4s/step - accuracy: 0.0537 - loss: 11.7383 - val_accuracy: 0.0338 - val_loss: 11.2054 - learning_rate: 1.0000e-05
Epoch 3/50
63/63 ━━━━━━━━━━━━━━━━━━━━ 223s 4s/step - accuracy: 0.1032 - loss: 11.3648 - val_accuracy: 0.0557 - val_loss: 11.1112 - learning_rate: 1.0000e-05
Epoch 4/50
63/63 ━━━━━━━━━━━━━━━━━━━━ 214s 3s/step - accuracy: 0.1483 - loss: 10.9261 - val_accuracy: 0.0835 - val_loss: 11.0044 - learning_rate: 1.0000e-05
Epoch 5/50
63/63 ━━━━━━━━━━━━━━━━━━━━ 228s 4s/step - accuracy: 0.2059 - loss: 10.6161 - val_accuracy: 0.1213 - val_loss: 10.9064 - learning_rate: 1.0000e-05
Epoch 6/50
63/63 ━━━━━━━━━━━━━━━━━━━━ 221s 3s/step - accuracy: 0.2605 - loss: 10.3742 - val_accuracy: 0.1392 - val_loss: 10.7975 - learning_rate: 1.0000e-05
Epoch 7/50
63/63 ━━━━━━━━━━━━━━━━━━━━ 224s 4s/step - accu

In [ ]:
model.save("/content/drive/MyDrive/sign_language_mobilenet1.h5")

In [ ]:
from google.colab import files

files.download("/content/drive/MyDrive/sign_language_mobilenet1.h5")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>